## 模型的保存与恢复
使用TensorFlow保存和恢复模型。  
这个示例使用了MNIST手写数字集(http://yann.lecun.com/exdb/mnist/)  

- Author: Aymeric Damien
- Project: https://github.com/aymericdamien/TensorFlow-Examples/

#### 1. 导入MNIST数据集

In [2]:
from __future__ import print_function

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data/", one_hot=True) #在当前目录下创建一个data文件夹即可

import tensorflow as tf

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


#### 2. 自定义参数

In [14]:
#超参数
learning_rate = 0.001
batch_size = 100
display_step = 1
model_path = '/Users/app/Documents/model_tf/model.ckpt'     #不能在当前py文件路径下

#网络模型参数
n_hidden_1 = 256     #第一层的特征数
n_hidden_2 = 256     #第二层的特征数
n_input = 784        #MNIST输入数据（图像形状：28*28像素）
n_classes = 10       #MNIST总的类别（0-9数字）

#TensorFlow的Graph输入
X = tf.placeholder('float',[None,n_input])
Y = tf.placeholder('float',[None,n_classes])

#定义weights和biases
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

#创建模型
def multilayer_percetron(x,weights,biases):
    #隐层，使用RELU激活函数
    layer_1 = tf.add(tf.matmul(x,weights['h1']),biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    #隐层，使用RELU激活函数
    layer_2 = tf.add(tf.matmul(layer_1,weights['h2']),biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    #输出层使用线性激活函数
    out_layer = tf.matmul(layer_2,weights['out']) + biases['out']
    return out_layer

pred = multilayer_percetron(X,weights,biases)

#定义loss和optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits= pred,
                                                             labels = Y))
optimizer = tf.train.AdadeltaOptimizer(learning_rate= learning_rate).minimize(cost)

#初始化变量
init = tf.global_variables_initializer()

#### 3. 模型保存

In [15]:
#saver操作用来保存和恢复所有的变量
saver = tf.train.Saver()

In [16]:
#运行第一个session
print ("运行第一个session...")
with tf.Session() as sess:
    #初始化变量
    sess.run(init)
    
    #训练循环
    for epoch in range(3):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples/batch_size)
        
        #遍历所有的batch
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            #运行optimizer操作（后向传播）和cost操作（得到loss值）
            _, c = sess.run([optimizer,cost],feed_dict = {X:batch_x,
                                                         Y:batch_y})
            #计算平均loss
            avg_cost += c / total_batch
        #显示每个epoch的loss
        if epoch % display_step == 0:
            print('Epoch:','%04d' %(epoch + 1),'cost =','{:.9f}'.format(avg_cost))
    print('训练结束！')
        
        
    #模型测试
    correct_prediction = tf.equal(tf.argmax(pred,1),tf.argmax(Y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    print('Accuarcy:',accuracy.eval({X: mnist.test.images, Y: mnist.test.labels}))
    
    
    
    #将模型参数保存到本地
    save_path = saver.save(sess,model_path)
    print ('模型保存在： %s' % save_path)

运行第一个session...
Epoch: 0001 cost = 1976.109488414
Epoch: 0002 cost = 1936.085860263
Epoch: 0003 cost = 1893.217941007
训练结束！
Accuarcy: 0.0925
模型保存在： /Users/app/Documents/model_tf/model.ckpt


#### 4. 模型恢复

In [22]:
#运行一个新的session
print ('运行第二个session...')
with tf.Session() as sess:
    #初始化变量
    sess.run(init)
    
    #恢复模型
    load_path = saver.restore(sess,model_path)
    print('模型恢复路径是： %s' %save_path)
    
    #恢复训练
    for epoch in range(7):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        #遍历所有的batch
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            #运行optimizer操作（后向传播）和cost操作（得到loss值）
            _, c = sess.run([optimizer,cost],feed_dict = {X:batch_x,
                                                         Y:batch_y})
            #计算平均loss
            avg_cost += c / total_batch
        #显示每个epoch的loss
        if epoch % display_step == 0:
            print('Epoch:','%04d' %(epoch + 1),'cost =','{:.9f}'.format(avg_cost))
    print('训练结束！')
        
        
    #模型测试
    correct_prediction = tf.equal(tf.argmax(pred,1),tf.argmax(Y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    print('Accuarcy:',accuracy.eval({X: mnist.test.images, Y: mnist.test.labels}))
    

运行第二个session...
INFO:tensorflow:Restoring parameters from /Users/app/Documents/model_tf/model.ckpt


INFO:tensorflow:Restoring parameters from /Users/app/Documents/model_tf/model.ckpt


模型恢复路径是： /Users/app/Documents/model_tf/model.ckpt
Epoch: 0001 cost = 1849.546460405
Epoch: 0002 cost = 1806.048778409
Epoch: 0003 cost = 1763.068796387
Epoch: 0004 cost = 1721.393671653
Epoch: 0005 cost = 1682.299213645
Epoch: 0006 cost = 1643.577783425
Epoch: 0007 cost = 1607.259913885
训练结束！
Accuarcy: 0.0924
